# 시장데이터 종가 생성

1. key 생성 : 거래소코드+년도
2. 종가변동률 구하기


In [270]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
plt.rcParams['font.family'] = 'malgun gothic'

from sklearn.metrics import accuracy_score, log_loss, plot_confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings(action='ignore')

## 데이터 불러오기

In [167]:
price = pd.read_csv(r'..\..\..\Code\SH\csv\0020종가.csv', encoding= 'euc-kr')

In [15]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가
0,(주)CMG제약,58820,2000/12,NaN,NaN
1,(주)CMG제약,58820,2001/12,0.00,1420.0
2,(주)CMG제약,58820,2002/12,0.00,520.0
3,(주)CMG제약,58820,2003/12,0.00,800.0
4,(주)CMG제약,58820,2004/12,0.00,550.0
...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0
27161,흥구석유(주),24060,2017/12,0.03,2775.0
27162,흥구석유(주),24060,2018/12,1.73,2705.0
27163,흥구석유(주),24060,2019/12,0.56,5360.0


In [17]:
price.isna().sum()

회사명              0
거래소코드            0
회계년도             0
외국인_주식분포비율    5369
종가            6379
dtype: int64

In [168]:
#일단 결측치 fillna로 0채워줌
price = price.fillna(0)

## key 생성
거래소코드+년도

In [169]:
price['회계년도'].str[:4]

0        2000
1        2001
2        2002
3        2003
4        2004
         ... 
27160    2016
27161    2017
27162    2018
27163    2019
27164    2020
Name: 회계년도, Length: 27165, dtype: object

In [170]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27165 entries, 0 to 27164
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   회사명         27165 non-null  object 
 1   거래소코드       27165 non-null  int64  
 2   회계년도        27165 non-null  object 
 3   외국인_주식분포비율  27165 non-null  float64
 4   종가          27165 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.0+ MB


In [171]:
price['년'] = price['회계년도'].str[:4].astype(int)

In [172]:
price['key'] = price['거래소코드'].astype(str) + (price['년']+1).astype(str)

In [173]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key
0,(주)CMG제약,58820,2000/12,0.00,0.0,2000,588202001
1,(주)CMG제약,58820,2001/12,0.00,1420.0,2001,588202002
2,(주)CMG제약,58820,2002/12,0.00,520.0,2002,588202003
3,(주)CMG제약,58820,2003/12,0.00,800.0,2003,588202004
4,(주)CMG제약,58820,2004/12,0.00,550.0,2004,588202005
...,...,...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0,2016,240602017
27161,흥구석유(주),24060,2017/12,0.03,2775.0,2017,240602018
27162,흥구석유(주),24060,2018/12,1.73,2705.0,2018,240602019
27163,흥구석유(주),24060,2019/12,0.56,5360.0,2019,240602020


## 종가변동률 구하기

In [174]:
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100

In [175]:
(len(company))

2020

In [176]:
(price.loc[price['회사명']==company[2019]]['종가'] - price.loc[price['회사명']==company[2019]]['종가'].shift(1))/price.loc[price['회사명']==company[2019]]['종가'].shift(1)*100
    

27144          NaN
27145   -12.452830
27146   -23.491379
27147   -17.746479
27148   -16.438356
27149    52.049180
27150    63.072776
27151    71.702479
27152   -99.534078
27153     9.917355
27154   -12.781955
27155   -35.344828
27156     5.666667
27157    26.182965
27158     8.250000
27159    40.415704
27160    12.664474
27161   -18.978102
27162    -2.522523
27163    98.151571
27164    34.701493
Name: 종가, dtype: float64

In [177]:
# for문으로 생성해주기
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100
company = price['회사명'].unique().tolist()

for i in tqdm(range(len(company))):
    price.loc[price['회사명']==company[i], '종가변동률']=(price.loc[price['회사명']==company[i]]['종가'] - price.loc[price['회사명']==company[i]]['종가'].shift(1))/price.loc[price['회사명']==company[i]]['종가'].shift(1)*100
    

100%|██████████| 2020/2020 [00:14<00:00, 135.53it/s]


In [178]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key,종가변동률
0,(주)CMG제약,58820,2000/12,0.00,0.0,2000,588202001,NaN
1,(주)CMG제약,58820,2001/12,0.00,1420.0,2001,588202002,inf
2,(주)CMG제약,58820,2002/12,0.00,520.0,2002,588202003,-63.380282
3,(주)CMG제약,58820,2003/12,0.00,800.0,2003,588202004,53.846154
4,(주)CMG제약,58820,2004/12,0.00,550.0,2004,588202005,-31.250000
...,...,...,...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0,2016,240602017,12.664474
27161,흥구석유(주),24060,2017/12,0.03,2775.0,2017,240602018,-18.978102
27162,흥구석유(주),24060,2018/12,1.73,2705.0,2018,240602019,-2.522523
27163,흥구석유(주),24060,2019/12,0.56,5360.0,2019,240602020,98.151571


In [179]:
price.loc[price['회사명']==company[10]]

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key,종가변동률
129,(주)고영테크놀러지,98460,2005/12,0.00,0.0,2005,984602006,NaN
130,(주)고영테크놀러지,98460,2006/12,0.00,0.0,2006,984602007,NaN
131,(주)고영테크놀러지,98460,2007/12,0.00,0.0,2007,984602008,NaN
132,(주)고영테크놀러지,98460,2008/12,4.24,6990.0,2008,984602009,inf
133,(주)고영테크놀러지,98460,2009/12,0.75,10700.0,2009,984602010,53.075823
134,(주)고영테크놀러지,98460,2010/12,5.41,18000.0,2010,984602011,68.224299
135,(주)고영테크놀러지,98460,2011/12,23.39,30900.0,2011,984602012,71.666667
136,(주)고영테크놀러지,98460,2012/12,17.01,28100.0,2012,984602013,-9.061489
137,(주)고영테크놀러지,98460,2013/12,17.83,27950.0,2013,984602014,-0.533808
138,(주)고영테크놀러지,98460,2014/12,38.70,42800.0,2014,984602015,53.130590


In [215]:
price['종가변동률'] = price['종가변동률'].replace([np.inf], np.nan)

In [216]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key,종가변동률
0,(주)CMG제약,58820,2000/12,0.00,0.0,2000,588202001,0.000000
1,(주)CMG제약,58820,2001/12,0.00,1420.0,2001,588202002,NaN
2,(주)CMG제약,58820,2002/12,0.00,520.0,2002,588202003,-63.380282
3,(주)CMG제약,58820,2003/12,0.00,800.0,2003,588202004,53.846154
4,(주)CMG제약,58820,2004/12,0.00,550.0,2004,588202005,-31.250000
...,...,...,...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0,2016,240602017,12.664474
27161,흥구석유(주),24060,2017/12,0.03,2775.0,2017,240602018,-18.978102
27162,흥구석유(주),24060,2018/12,1.73,2705.0,2018,240602019,-2.522523
27163,흥구석유(주),24060,2019/12,0.56,5360.0,2019,240602020,98.151571


In [218]:
# 일단 결측치 0으로 채워줌
price = price.fillna(0)
price.isna().sum()

회사명           0
거래소코드         0
회계년도          0
외국인_주식분포비율    0
종가            0
년             0
key           0
종가변동률         0
dtype: int64

## finaldataset과 merge

In [219]:
df_ = pd.read_csv(r'..\..\..\Data\최종데이터셋\Finaldataset_ver6.csv')
df = df_

In [220]:
df['key']=df['key'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26832 entries, 0 to 26831
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   회사명          26832 non-null  object 
 1   거래소코드        26832 non-null  float64
 2   회계년도         26832 non-null  object 
 3   소속코드         26832 non-null  float64
 4   상장폐지일        26832 non-null  object 
 5   감사의견코드       26832 non-null  object 
 6   자기자본배당률      26832 non-null  float64
 7   금융비용 대 부채비율  26832 non-null  float64
 8   자기자본배율       26832 non-null  float64
 9   유동부채비율       26832 non-null  float64
 10  매입채무회전률      26832 non-null  float64
 11  유동자산회전률      26832 non-null  float64
 12  총자본투자효율      26832 non-null  float64
 13  순운전자본비율      26832 non-null  float64
 14  누적수익성비율      26832 non-null  float64
 15  총자산영업이익률     26832 non-null  float64
 16  총자산회전율       26832 non-null  float64
 17  년            26832 non-null  float64
 18  key          26832 non-null  object 
 19  부도  

In [221]:
price.columns
price_col = [ '외국인_주식분포비율', '종가', 'key', '종가변동률']

In [222]:
print(df_.shape)
print(df.shape)

(26832, 28)
(26832, 28)


In [223]:
#merge 진행
df = df.merge(price[price_col], how = 'outer', on = 'key')

#해당되지 않는 데이터 삭제를 위해 fillna 처리
df['횡령배임'] = df['횡령배임'].fillna(0)

#해당되지 않는 데이터 삭제 
df.dropna(inplace=True)
df.drop(columns=['폐지사유요약합본'], inplace=True)
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,공시발생일,정기공시제목,수시공시제목,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,외국인_주식분포비율,종가,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,0.00,1420.0,0.000000
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,0,0,0,1.0,0.0,0.0,0.0,0.00,520.0,-63.380282
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,0,0,0,1.0,0.0,0.0,0.0,0.00,800.0,53.846154
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,0,0,0,1.0,0.0,0.0,0.0,0.00,550.0,-31.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26827,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,0,0,0,0.0,0.0,0.0,0.0,0.97,3425.0,12.664474
26828,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,0,0,0,0.0,0.0,0.0,0.0,0.03,2775.0,-18.978102
26829,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,0,0,0,0.0,0.0,0.0,0.0,1.73,2705.0,-2.522523
26830,흥구석유(주),24060.0,2019/12,5.0,0,UQ,2.83,0.000300,5133.42,6.05,...,0,0,0,0.0,0.0,0.0,0.0,0.56,5360.0,98.151571


In [224]:
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,공시발생일,정기공시제목,수시공시제목,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,외국인_주식분포비율,종가,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,0.00,1420.0,0.000000
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,0,0,0,1.0,0.0,0.0,0.0,0.00,520.0,-63.380282
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,0,0,0,1.0,0.0,0.0,0.0,0.00,800.0,53.846154
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,0,0,0,1.0,0.0,0.0,0.0,0.00,550.0,-31.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26827,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,0,0,0,0.0,0.0,0.0,0.0,0.97,3425.0,12.664474
26828,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,0,0,0,0.0,0.0,0.0,0.0,0.03,2775.0,-18.978102
26829,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,0,0,0,0.0,0.0,0.0,0.0,1.73,2705.0,-2.522523
26830,흥구석유(주),24060.0,2019/12,5.0,0,UQ,2.83,0.000300,5133.42,6.05,...,0,0,0,0.0,0.0,0.0,0.0,0.56,5360.0,98.151571


## 데이터 저장

In [225]:
#df.to_csv('dataset with 종가변동률.csv')

# 모델링

## 데이터 생성 : 재무+비재무 / only 재무

In [226]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '상장폐지일', '감사의견코드', '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율', '년', 'key', '부도', '공시발생일',
       '정기공시제목', '수시공시제목', '대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임',
       '외국인_주식분포비율', '종가', '종가변동률'],
      dtype='object')

In [227]:
#재무+비재무+시장데이터
col_f_t = [ '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율', '대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임',
       '외국인_주식분포비율', '종가변동률'] #종가는 너무 커서 아직 안넣음 추후 로그 스케일링 진행예정

#재무데이터
col_f =  [ '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율']

In [228]:
#feature finance+text
X_ft = df[col_f_t]
#feature only finance
X_fin = df[col_f]

y=df['부도']

In [229]:
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
26827    1.0
26828    1.0
26829    1.0
26830    1.0
26831    1.0
Name: 부도, Length: 26832, dtype: float64

In [230]:
X_ft

,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,매입채무회전률,유동자산회전률,총자본투자효율,순운전자본비율,누적수익성비율,총자산영업이익률,총자산회전율,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,외국인_주식분포비율,종가변동률
0,0.00,0.019074,179.37,131.99,24.84,0.97,8.76,22.76,0.189100,0.052949,1.761228,0.0,0.0,0.0,0.0,0.00,0.000000
1,0.00,0.023517,179.37,131.99,24.84,0.97,8.76,22.76,0.005261,0.014062,0.756792,0.0,0.0,0.0,0.0,0.00,0.000000
2,0.00,0.043081,133.37,132.37,58.88,2.01,0.79,7.98,-0.139212,-0.247841,1.713785,1.0,0.0,0.0,0.0,0.00,-63.380282
3,0.00,0.058531,131.30,71.46,26.59,1.89,16.80,35.58,-0.203019,-0.082618,1.571816,1.0,0.0,0.0,0.0,0.00,53.846154
4,0.00,0.047346,52.25,101.56,24.16,2.11,-37.94,24.97,-1.181502,-0.547982,2.526356,1.0,0.0,0.0,0.0,0.00,-31.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26827,2.10,0.012651,4611.84,5.19,179576.67,10.07,8.84,14.15,0.414800,0.012181,1.817620,0.0,0.0,0.0,0.0,0.97,12.664474
26828,2.08,0.012502,4650.29,12.83,5044.25,10.14,8.80,8.68,0.417422,0.009398,1.946103,0.0,0.0,0.0,0.0,0.03,-18.978102
26829,2.48,0.018056,4696.15,5.40,5128.70,11.71,9.80,11.59,0.453302,0.014520,2.210036,0.0,0.0,0.0,0.0,1.73,-2.522523
26830,2.83,0.000300,5133.42,6.05,23497.93,8.58,18.02,21.47,0.485889,0.005988,1.800947,0.0,0.0,0.0,0.0,0.56,98.151571


In [231]:
X_fin

,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,매입채무회전률,유동자산회전률,총자본투자효율,순운전자본비율,누적수익성비율,총자산영업이익률,총자산회전율
0,0.00,0.019074,179.37,131.99,24.84,0.97,8.76,22.76,0.189100,0.052949,1.761228
1,0.00,0.023517,179.37,131.99,24.84,0.97,8.76,22.76,0.005261,0.014062,0.756792
2,0.00,0.043081,133.37,132.37,58.88,2.01,0.79,7.98,-0.139212,-0.247841,1.713785
3,0.00,0.058531,131.30,71.46,26.59,1.89,16.80,35.58,-0.203019,-0.082618,1.571816
4,0.00,0.047346,52.25,101.56,24.16,2.11,-37.94,24.97,-1.181502,-0.547982,2.526356
...,...,...,...,...,...,...,...,...,...,...,...
26827,2.10,0.012651,4611.84,5.19,179576.67,10.07,8.84,14.15,0.414800,0.012181,1.817620
26828,2.08,0.012502,4650.29,12.83,5044.25,10.14,8.80,8.68,0.417422,0.009398,1.946103
26829,2.48,0.018056,4696.15,5.40,5128.70,11.71,9.80,11.59,0.453302,0.014520,2.210036
26830,2.83,0.000300,5133.42,6.05,23497.93,8.58,18.02,21.47,0.485889,0.005988,1.800947


In [247]:
def modeling(model):
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    y_true = y_test
    #y_pred = (pred[:,0] < 0.3) * 1    #임계값 조절 0.025~0.3

    return y_pred

## 재무+비재무+시장정보 : X_ft

In [233]:
X_train, X_test, y_train, y_test = train_test_split(X_ft, y, test_size=0.3, stratify=y, random_state=46)

In [271]:
rf_clf=RandomForestClassifier(criterion='entropy')

rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
y_true = y_test
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))
print('Recall score : ', recall_score(y_true, y_pred, average='weighted'))
print('precision score : ', precision_score(y_true, y_pred, average='weighted'))
print('f1_score : ', f1_score(y_true, y_pred, average='weighted'))

[[  12  109]
 [  17 7912]]
Recall score :  0.9843478260869565
precision score :  0.9778036253036677
f1_score :  0.9795929671527872


In [264]:
ada_clf = AdaBoostClassifier()

ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
y_true = y_test
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))
print('Recall score : ', recall_score(y_true, y_pred, average='weighted'))
print('precision score : ', precision_score(y_true, y_pred, average='weighted'))
print('f1_score : ', f1_score(y_true, y_pred, average='weighted'))

[[  21  100]
 [  29 7900]]
Recall score :  0.9839751552795031
precision score :  0.9789698757763976
f1_score :  0.9806840736338496


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


## only 재무데이터

In [273]:
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size=0.3, stratify=y, random_state=46)

In [274]:
rf_clf=RandomForestClassifier(criterion='entropy')

rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
y_true = y_test
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))
print('Recall score : ', recall_score(y_true, y_pred, average='weighted'))
print('precision score : ', precision_score(y_true, y_pred, average='weighted'))
print('f1_score : ', f1_score(y_true, y_pred, average='weighted'))

[[  11  110]
 [  20 7909]]
Recall score :  0.9838509316770186
precision score :  0.9767913107791555
f1_score :  0.979115524850993


In [275]:
ada_clf = AdaBoostClassifier()

ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
y_true = y_test
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))
print('Recall score : ', recall_score(y_true, y_pred, average='weighted'))
print('precision score : ', precision_score(y_true, y_pred, average='weighted'))
print('f1_score : ', f1_score(y_true, y_pred, average='weighted'))

[[  12  109]
 [  21 7908]]
Recall score :  0.9838509316770186
precision score :  0.9770430382022054
f1_score :  0.9792814724851654
